<a href="https://colab.research.google.com/github/ZiyangZ/Machine-learning/blob/master/HW1_problem5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.datasets import mnist
from scipy import ndimage
from keras import models
from keras import layers
from keras.utils import to_categorical
import numpy as np

#load data
(train_images_original, train_labels_original), (test_images_original, test_labels_original) = mnist.load_data()

train_images = train_images_original.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images_original.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels_original)
test_labels = to_categorical(test_labels_original)

#change each image into black/white
tr = train_images_original.astype('float32') / 255
tr[tr<0.5]=0
tr[tr>=0.5]=1
bw_images_train = 1-tr

te = test_images_original.astype('float32') / 255
te[te<0.5]=0
te[te>=0.5]=1
bw_images_test = 1-te

#calculate normalized white regions using connected components
white_region_number_train = np.zeros((60000,1))
for i in range(60000):
  w1 = bw_images_train[i]
  white_region_train, white_region_train = ndimage.label(w1)
  white_region_number_train[i]=white_region_train/3 

white_region_number_test = np.zeros((10000,1))
for i in range(10000):
  w2 = bw_images_test[i]
  white_region_test, white_region_test = ndimage.label(w2)
  white_region_number_test[i]=white_region_test/3
  
#add the new feature to original features
train_images_new = np.concatenate((train_images, white_region_number_train), axis = 1)
test_images_new = np.concatenate((test_images, white_region_number_test), axis = 1)

#build neural network
network = models.Sequential()
network.add(layers.Dense(10, activation='softmax', input_shape=(28 * 28 + 1,))) 

network.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

epochs = 20
history = network.fit(train_images_new, 
                      train_labels, 
                      epochs=epochs, 
                      batch_size=50, 
                      validation_data=(test_images_new, test_labels))